In [1]:
# %%capture
!pip install labml labml_nn labml_helpers labml_python_autocomplete

     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 276kB 50.3MB/s 
     |████████████████████████████████| 163kB 48.1MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=0671b36fedde7d40f7d54cad93cf8ad8a0a436bc6e475e2ee43d792552d37460
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from python_autocomplete import create_dataset
from labml.logger import inspect
from labml import monit, lab

import numpy as np

In [3]:
create_dataset.create_folders()

Get the list of repositories from [Awesome-PyTorch-list](https://github.com/bharathgs/Awesome-pytorch-list/)

In [4]:
create_dataset.get_awesome_pytorch_readme()
repos = create_dataset.get_repos_from_readme('pytorch_awesome.md')
inspect(repos)

Download zip files. For demonstration we only use 10 repos.

In [5]:
repos = repos[:10]
for i, r in monit.enum(f"Download {len(repos)} repos", repos):
    zip_file = create_dataset.download_repo(r[0], r[1], i)
    extracted = create_dataset.extract_zip(zip_file)
    create_dataset.remove_files(extracted, {'.py'})

In [6]:
source_files = create_dataset.get_python_files()
np.random.shuffle(source_files)
inspect(source_files)

In [7]:
train_valid_split = int(len(source_files) * 0.9)
create_dataset.concat_and_save(lab.get_data_path() / 'train.py', source_files[:train_valid_split])
create_dataset.concat_and_save(lab.get_data_path() / 'valid.py', source_files[train_valid_split:])

In [8]:
from python_autocomplete.train import Configs
from labml import experiment

In [9]:
conf = Configs()
experiment.create(name="python_autocomplete",
                  comment='lstm model')

Set configurations

In [10]:
experiment.configs(conf, {
    'model': 'lstm_model',  # Model
    'n_layers': 2,  # Number of layers
    'batch_size': 64,  # Batch size
    'epochs': 32,   # Number of epochs
    'optimizer.optimizer': 'Adam',  # Optimizer
    'optimizer.learning_rate': 2.5e-4,  # Learning rate
    'device.cuda_device': 0,  # Device
    'inner_iterations': 5  # Number of iterations within an epoch 
                           # (switches between training and validation)
})

In [11]:
experiment.add_pytorch_models({'model': conf.model})

In [ ]:
# experiment.load('d5ba7f56d88911eaa6629b54a83956dc')
with experiment.start():
    conf.run()